## 简单的问答示例

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [6]:
context = """
1.可口可乐价格4元
2.百事可乐价格4.5元
3.雪碧价格3.5元
4.脉动价格6元
5.农夫山泉价格2元
6.果粒橙价格5元

"""

question = "从中找一个最贵的饮料"

In [8]:
prompt = context + question

chat([
    SystemMessage(content=prompt)
])

AIMessage(content='脉动价格6元是最贵的饮料。', additional_kwargs={}, example=False)

## 使用嵌入（长文本）

随着复杂性的提高，我们将更多的利用这个约定。当你需要选择将哪些数据放入上下文中时，困难的部分就出现了。这个研究领域被称为 “文档检索”，并于 AI Memory 紧密结合。


In [16]:
from langchain.chat_models import ChatOpenAI
# 向量存储
from langchain.vectorstores import FAISS
# 问答链
from langchain.chains import RetrievalQA
# 文本文档加载器
from langchain.document_loaders import TextLoader
# 文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 嵌入引擎：将文本转为向量
from langchain.embeddings.openai import OpenAIEmbeddings

chat = ChatOpenAI()

In [51]:
# 创建文档加载器
loader = TextLoader('data/vul-scan.txt', encoding='utf-8')# https://www.intruder.io/guides/the-ultimate-guide-to-vulnerability-scanning
# 加载文档
documents = loader.load()

print(f"doc num: {len(documents)}\ncharacter num of all: {[len(doc.page_content) for doc in documents]}")

doc num: 1
character num of all: [40556]


In [52]:
# 创建分割器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
# 使用分割器进行分割
splitted_docs = text_splitter.split_documents(documents)

len_arr = [len(d.page_content) for d in splitted_docs]
print(f"length of all splitted text segment: {len_arr}\navg length: {sum(len_arr) / len(len_arr)}")

length of all splitted text segment: [1875, 1938, 1273, 1957, 1780, 1852, 1573, 1992, 1884, 1571, 1671, 1523, 1849, 1399, 1927, 1737, 1374, 1676, 1550, 1759, 1991, 1757, 1732, 1507]
avg length: 1714.4583333333333


In [53]:
# 创建嵌入引擎
embeddings = OpenAIEmbeddings()
# 嵌入文档并与为数据库中的原始文本结合。会调用 OpenAI 的 API
doc_search = FAISS.from_documents(splitted_docs, embeddings)

In [54]:
# 创建检索引擎。检索器会获取类似的文本，并结合问题让 LLM 进行推理。
qa = RetrievalQA.from_chain_type(chat, chain_type='stuff', retriever=doc_search.as_retriever())

In [58]:
print(qa.run('漏洞扫描器类型有哪些?'))

主要的漏洞扫描器类型包括：

1. 网络漏洞扫描器：用于检测网络设备、服务器和操作系统上的漏洞，包括已知的安全漏洞和配置错误。

2. Web应用程序漏洞扫描器：专注于发现Web应用程序和网站中的漏洞，例如SQL注入、跨站脚本等。

3. 主机漏洞扫描器：用于检测主机上的操作系统和应用程序的漏洞，包括缺少安全补丁、弱密码等。

4. 移动应用程序漏洞扫描器：专门用于检测移动应用程序中的漏洞，例如不安全的数据存储、不正确的权限控制等。

5. 数据库漏洞扫描器：用于检测数据库服务器上的漏洞，例如未授权访问、弱密码等。

6. 工控系统漏洞扫描器：专门用于检测工业控制系统中的漏洞，例如SCADA系统、PLC等。

请注意，不同的漏洞扫描器可能具有不同的功能和特点，你需要根据自己的需求选择适合的扫描器。


In [57]:
print(qa.run('文中推荐的漏洞扫描工具？'))

文中并没有具体推荐扫描工具，但文中提到了Intruder，他们提供了免费试用的在线漏洞扫描工具。您可以通过访问他们的网站来了解更多信息。


In [59]:
print(qa.run("漏洞扫描工具适合哪些人？"))

漏洞扫描工具适用于所有规模的企业和组织。无论是大型企业还是中小型企业，都应该考虑到网络安全威胁，并制定相应的安全策略。最近英国政府的一项调查显示，近两年中有61%的中大型企业和38%的小型企业都遭受了网络攻击或数据泄露。因此，无论企业规模大小，都应该认识到存在的网络安全威胁，并采取相应的措施。

漏洞扫描工具可以帮助组织发现最严重、最容易受攻击的技术弱点，以便在攻击者利用之前采取措施。对于所有企业来说，了解其网络安全漏洞的位置并进行修复都是非常重要的。


## 问答结果评估

In [73]:
from langchain.evaluation.qa import QAEvalChain

# 给定问题和答案1，将这里的答案视为“标准答案”
question_answers = [
    {'question': '漏洞扫描工具适合哪些人', 'answer': '漏洞扫描工具仅适用于个人'},
    {'question': '文中推荐的漏洞扫描工具', 'answer': '文中没有推荐漏洞扫描工具'}
]

In [74]:
chain = RetrievalQA.from_chain_type(chat, 
                                    chain_type='stuff', 
                                    retriever=doc_search.as_retriever(), 
                                    input_key="question") # 指定【输入键】

# 进行问答，得到答案2，将这里的答案视为“参考答案”
predictions = chain.apply(question_answers)
predictions

[{'question': '漏洞扫描工具适合哪些人',
  'answer': '漏洞扫描工具仅适用于个人',
  'result': '漏洞扫描工具适合所有规模的企业和组织。尽管新闻头条通常关注影响大型组织的最大安全漏洞，但这并不意味着小型企业就安全了。根据英国政府最近的一项调查，近三分之二（61%）的中大型企业在过去12个月中发现了网络攻击。小型企业也不例外，其中38%报告了攻击或数据泄露事件。这些调查结果表明，各种规模的企业都应该考虑存在的威胁，并制定一套能够应对这些威胁的安全策略。\n\n几乎所有规模的企业都依赖技术来运营，无论是通过网站提供营销或博客内容，还是运行面向互联网的应用程序或服务，甚至是员工用于工作的笔记本电脑。所有这些系统都构成了黑客攻击的目标。成功的入侵可能导致勒索软件攻击，甚至是对不太敏感的数据（如姓名和地址）的泄露，这可能导致客户流失或面临重罚款。\n\n因此，每个企业都应该了解其网络漏洞所在，并及时修复。漏洞扫描工具可以帮助企业识别其最严重和最容易受攻击的技术漏洞，以便能够在攻击者利用之前采取行动。'},
 {'question': '文中推荐的漏洞扫描工具',
  'answer': '文中没有推荐漏洞扫描工具',
  'result': '在这篇文章中没有具体提到或推荐任何特定的漏洞扫描工具。文章的目的是提供有关漏洞扫描的最佳实践和功能的信息，以帮助读者选择适合自己需求的漏洞扫描工具。读者可以根据自己的需求和预算来选择适合自己的漏洞扫描工具。'}]

In [76]:
# 创建评估链
eval_chain = QAEvalChain.from_llm(chat)

# 进行评估。评估 LLM回复的结果 与指定的结果 的相似度
eval_chain.evaluate(question_answers,
                   predictions,
                   question_key="question",
                   answer_key="answer",
                   prediction_key="result")

[{'results': 'INCORRECT'}, {'results': 'CORRECT'}]